In [2]:
import numpy as np
import pandas as pd



url = "https://archivo.datos.cdmx.gob.mx/fiscalia-general-de-justicia/victimas-en-carpetas-de-investigacion-fgj/da_victimas_completa_marzo_2023.csv"
data_cdmx = pd.read_csv(url)

#sort out delicts that happened outside of cdmx
boolean_mask = data_cdmx['municipio_hechos'].isnull()
data = data_cdmx[boolean_mask].reset_index(drop=True)

#sort out crimes that were not commited against persons but companies
boolean_mask = data['TipoPersona'] == 'FISICA'
data = data[boolean_mask].reset_index(drop=True)

#columns to drop
columns_to_drop = ['idCarpeta', 'Año_inicio', 'Mes_inicio', 'FechaInicio',
                   'CalidadJuridica', 'HoraInicio',
                  'fgj_colonia_registro', 'municipio_hechos']

data = data.drop(columns_to_drop, axis=1)

## DATA CLEANING

# Remove duplicate rows
data = data.drop_duplicates()


# Remove Ano Hecho below 2019
data = data[data["Año_hecho"] > 2018]


# Remove age higher than 100 and replace all missing ages by mean age
data['Edad'] = np.where(data['Edad'] > 100, np.nan, data['Edad'])

data['Edad'] = np.where(data['Edad'].isna(), data["Edad"].mean(), data['Edad'])


# Remove rows with missing gender
data = data[data["Sexo"].notna()]


# Assign missing latitude and longitude, set wrong longitudes to 0
data['latitud'] = np.where(data['latitud'].isna(), 0, data['latitud'])

data['longitud'] = np.where(data['longitud'].isna(), 0, data['longitud'])

data['longitud'] = np.where(data['longitud'] == data["latitud"], 0, data['longitud'])

# First Visualization attempt

In [3]:
import folium 
import pandas as pd
from IPython.display import display
import ipywidgets as widgets



In [4]:
#Creating a subset of the dataset 

map_data = data.sample(n=100, random_state=42)
map_data.head(5)

,Delito,Categoria,Sexo,Edad,TipoPersona,competencia,Año_hecho,Mes_hecho,FechaHecho,HoraHecho,alcaldia_hechos,colonia_datos,latitud,longitud
96926,HOMICIDIO CULPOSO POR TRÁNSITO VEHICULAR (ATRO...,DELITO DE BAJO IMPACTO,Femenino,24.000000,FISICA,FUERO COMUN,2019.0,Abril,2019-04-30,22:00:00,GUSTAVO A. MADERO,TEPEYAC INSURGENTES,19.489300,-99.121090
179209,VIOLENCIA FAMILIAR,DELITO DE BAJO IMPACTO,Masculino,38.984155,FISICA,FUERO COMUN,2019.0,Julio,2019-07-25,18:00:00,TLAHUAC,LAS ARBOLEDAS,19.303050,-99.053650
342015,AMENAZAS,DELITO DE BAJO IMPACTO,Masculino,44.000000,FISICA,FUERO COMUN,2020.0,Noviembre,2020-11-17,22:19:00,GUSTAVO A. MADERO,LINDAVISTA II,19.495033,-99.123544
97493,ABUSO SEXUAL,DELITO DE BAJO IMPACTO,Femenino,23.000000,FISICA,FUERO COMUN,2019.0,Abril,2019-04-26,16:58:00,CUAUHTEMOC,VISTA ALEGRE,19.409100,-99.135400
327770,PERSONAS EXTRAVIADAS Y AUSENTES,DELITO DE BAJO IMPACTO,Femenino,10.000000,FISICA,FUERO COMUN,2020.0,Octubre,2020-10-29,19:10:00,GUSTAVO A. MADERO,SANTA ROSA,19.525039,-99.173015


In [26]:
map_data.Categoria.value_counts()

Categoria
DELITO DE BAJO IMPACTO                                   87
ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA      5
ROBO DE VEHÍCULO CON Y SIN VIOLENCIA                      3
VIOLACIÓN                                                 2
ROBO A PASAJERO A BORDO DEL METRO CON Y SIN VIOLENCIA     1
HOMICIDIO DOLOSO                                          1
HECHO NO DELICTIVO                                        1
Name: count, dtype: int64

In [35]:
mexico_city = folium.Map(location=[19.4326, -99.1332], zoom_start=11, tiles= 'Stamen Toner')
filtered_mexico_city = folium.Map(location=[19.4326, -99.1332], zoom_start=11, tiles= 'Stamen Toner')


In [30]:
# Function to add markers
def add_marker(row, map):
    folium.Marker(
        location=[row['latitud'], row['longitud']],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(map)

# Apply the add_marker function to each row of the DataFrame
map_data.apply(add_marker, map=mexico_city, axis=1)

mexico_city

In [38]:
def search_keyword(keyword, map):
    map = folium.Map(location=[19.4326, -99.1332], zoom_start=11, tiles= 'Stamen Toner')
    filtered_df = map_data[map_data['Categoria'].str.contains(keyword, case=False)]
    filtered_df.apply(add_marker, map=map, axis=1)
    return map

# Example usage: Search for 'category_keyword' in 'category_column'
search_keyword('DELITO DE BAJO IMPACTO', filtered_mexico_city)